# Example of computing vibrational energy levels of a triatomic molecule using the Taylor series expansion approximation for the kinetic and potential energy operators

Compute vibrational states of $\text{H}_2\text{S}$ molecule using the potential energy surface from
[A. A. A. Azzam, J. Tennyson, S. N. Yurchenko, O. V. Naumenko, "ExoMol molecular line lists - XVI. The rotation–vibration spectrum of hot H2S", MNRAS 460, 4063–4074 (2016)](https://doi.org/10.1093/mnras/stw1133)


In [2]:
import sys

sys.path.insert(1, "../")

from typing import Callable, List

import jax
import jax.numpy as jnp
import numpy as np
from numpy.polynomial.hermite import hermder, hermgauss, hermval
from scipy import optimize
from scipy.special import factorial
from vibrojet.keo import Gmat, com
from vibrojet.potentials import h2s_AYT2
from vibrojet.taylor import deriv_list

jax.config.update("jax_enable_x64", True)

Compute equilibrium coordinates, around which the Taylor series expansions will be built

In [3]:
vmin = optimize.minimize(h2s_AYT2.poten, [1, 1, 90 * np.pi / 180])
r0 = vmin.x
v0 = vmin.fun
print("Equilibrium coordinates:", r0)
print("Min of the potential:", v0)

Equilibrium coordinates: [1.3358387  1.3358387  1.61042427]
Min of the potential: -0.0007846164036985615


Define mapping function from internal valence coordinates, $r_1$, $r_2$, and $\alpha$, to Cartesian coordinates

In [4]:
masses = [31.97207070, 1.00782505, 1.00782505]  # masses of S, H, H

ncoo = len(r0)


@com(masses)
def internal_to_cartesian(coords):
    r1, r2, alpha = coords
    return jnp.array(
        [
            [0.0, 0.0, 0.0],
            [r1 * jnp.cos(alpha / 2), 0.0, r1 * jnp.sin(alpha / 2)],
            [r2 * jnp.cos(alpha / 2), 0.0, -r2 * jnp.sin(alpha / 2)],
        ]
    )

Utility function for generating index combinations corresponding to products of primitive basis functions as well as derivative indices used in the expansion of operators

In [5]:
def generate_prod_ind(
    indices: List[List[int]],
    select: Callable[[List[int]], bool] = lambda _: True,
    batch_size: int = None,
):
    no_elem = np.array(
        [len(elem) for elem in indices]
    )  # Highest order (z) + 1  - [x^0,x^1,...,x^z] -> z+1 terms
    tot_size = np.prod(
        no_elem
    )  # Total possible number of combinations (z_0*z_1*...*z_N)
    if batch_size is None:
        batch_size = tot_size
    # no_batches = (tot_size + batch_size - 1) // batch_size
    no_batches = int(np.ceil(tot_size / batch_size))
    for ibatch in range(no_batches):  # Loop over batches
        start_ind = ibatch * batch_size  # Start index from end index of previous batch.
        end_ind = np.minimum(start_ind + batch_size, tot_size)
        batch_ind = np.arange(start_ind, end_ind)  # Counting index
        multi_ind = np.array(
            np.unravel_index(batch_ind, no_elem)
        )  # N-dimensional indexes (N x 1D basis indexes)
        indices_out = np.array(
            [indices[icoo][multi_ind[icoo, :]] for icoo in range(len(indices))]
        ).T  # All possible N-dimensional indexes
        select_ind = np.where(
            np.asarray([select(elem) for elem in indices_out])
        )  # Truncation for combinations
        yield indices_out[select_ind], multi_ind[
            :, select_ind[0]
        ]  # Output indexes that adhere to the select truncation

Compute Taylor series expansion of the $G$-matrix and potential

$$
G_{\lambda,\mu}=\sum_{\mathbf{t}} g_{\mathbf{t},\lambda,\mu} (r_1-r_1^\text{(eq)})^{t_1}(r_2-r_2^\text{(eq)})^{t_2}(\alpha-\alpha^\text{(eq)})^{t_3},
$$

$$
V=\sum_{\mathbf{t}} f_{\mathbf{t}} (r_1-r_1^\text{(eq)})^{t_1}(r_2-r_2^\text{(eq)})^{t_2}(\alpha-\alpha^\text{(eq)})^{t_3},
$$

 where the derivative multi-index $\mathbf{t}$ is stored in `deriv_ind`, and the expansion coefficients $g_{\mathbf{t},\lambda,\mu}$ and $f_{\mathbf{t}}$ in `Gmat_coefs[:len(deriv_ind),:3N, :3N]` and `poten_coefs[:len(deriv_ind)]` respectively, where $N$ is the number of atoms


In [6]:
max_pow = 8  # Degree of Taylor Expansion
powers = [np.arange(max_pow + 1)] * ncoo  # 1D possible powers
tot_size = np.prod([len(p) for p in powers])  # Number of terms before truncation
batch_size = tot_size  # batch size - tot_size => one batch
no_batches = int(np.ceil(tot_size / batch_size))
select = lambda ind: np.sum(ind) <= max_pow  # Truncation criterion for Taylor expansion
gen_prod_ind = generate_prod_ind(
    powers, select=lambda ind: np.sum(ind) <= max_pow, batch_size=batch_size
)
deriv_ind_list, deriv_mind_list = [], []  # Create empty lists to append to
for i in range(no_batches):  # Loop over batches
    _deriv_ind, _deriv_mind = next(gen_prod_ind)
    deriv_ind_list.append(_deriv_ind)  # Append indexes
    deriv_mind_list.append(_deriv_mind)

# Concatenate lists of indexes
deriv_ind, deriv_mind = np.concatenate(deriv_ind_list, axis=0), np.concatenate(
    deriv_mind_list, axis=1
)

print("Max expansion power:", max_pow)
print("Total number of terms before truncation:", tot_size)
print("Number of expansion terms after truncation:", len(deriv_ind))

Max expansion power: 8
Total number of terms before truncation: 729
Number of expansion terms after truncation: 165


In [7]:
print("Compute expansion of potential ...")
poten_coefs = deriv_list(h2s_AYT2.poten, deriv_ind, r0, if_taylor=True)

Compute expansion of potential ...
Time for d= 0 : 20.86 s
Time for d= 1 : 16.78 s
Time for d= 2 : 40.04 s
Time for d= 3 : 61.08 s
Time for d= 4 : 88.68 s
Time for d= 5 : 68.3 s
Time for d= 6 : 92.64 s
Time for d= 7 : 114.4 s
Time for d= 8 : 131.01 s


In [8]:
print("Compute expansion of G-matrix ...")
Gmat_coefs = deriv_list(
    lambda x: Gmat(x, masses, internal_to_cartesian), deriv_ind, r0, if_taylor=True
)

Compute expansion of G-matrix ...
Time for d= 0 : 0.97 s
Time for d= 1 : 0.75 s
Time for d= 2 : 1.78 s
Time for d= 3 : 2.96 s
Time for d= 4 : 4.96 s
Time for d= 5 : 8.09 s
Time for d= 6 : 8.85 s
Time for d= 7 : 11.48 s
Time for d= 8 : 14.33 s


Define primitive Hermite basis functions $\mathcal{H}_n(x)$ and their derivatives $d\mathcal{H}_n(x)/dx$

In [15]:
def hermite(x, n):
    sqsqpi = np.sqrt(np.sqrt(np.pi))
    c = np.diag(1.0 / np.sqrt(2.0**n * factorial(n)) / sqsqpi) #Normalization constants
    f = hermval(np.asarray(x), c) * np.exp(-(x**2) / 2) #Hermite polynomial times square root of weight function
    return f.T


def hermite_deriv(x, n):
    sqsqpi = np.sqrt(np.sqrt(np.pi))
    c = np.diag(1.0 / np.sqrt(2.0**n * factorial(n)) / sqsqpi) #Normalization constants
    h = hermval(np.asarray(x), c) #Hermite polynomial 
    dh = hermval(np.asarray(x), hermder(c, m=1)) #Derivative of Hermite polynomial (H_l(x))
    f = (dh - h * x) * np.exp(-(x**2) / 2) #w'(x)H_l(x) + w(x)H_l'(x), where w: weight function, w(x) = np.exp(-x**2)
    return f.T

Define mapping between the coordinate $x\in (-\infty, \infty)$ of Hermite basis functions and the internal valence coordinates as $r_1=a_1 x_1 + b_1$, $r_2=a_2x_2+b_2$, $\alpha=a_3x_3+b_3$. The parameters $a_1,b_1,...,b_3$ are determined by mapping the vibrational Hamiltonian in valence coordinates onto the harmonic-oscillator Hamiltonian.

In [16]:
mask = deriv_ind != 0  # Mask for derivatives

# Vibrational G-matrix elements at equilibrium geometry
ind0 = np.where(mask.sum(axis=1) == 0)[0][0] #Equilibrium indexes
mu = np.diag(Gmat_coefs[ind0])[:ncoo]

# Second-order derivative of potential at equilibrium
ind2 = np.array(
    [   np.where((mask.sum(axis=1) == 1) & (deriv_ind[:, icoo] == 2))[0][0]
        for icoo in range(ncoo)]
)

freq = poten_coefs[ind2] * 2

# Linear mapping parameters
lin_a = jnp.sqrt(jnp.sqrt(mu / freq))
lin_b = r0

print("x->r linear mapping parameters 'a':", lin_a)
print("x->r linear mapping parameters 'b':", lin_b)

# Linear mapping function and its derivative
x_to_r_map = lambda x, icoo: lin_a[icoo] * x + lin_b[icoo]
jac_x_to_r_map = lambda x, icoo: np.ones_like(x) * lin_a[icoo]

x->r linear mapping parameters 'a': [0.11245619 0.11245619 0.17845517]
x->r linear mapping parameters 'b': [1.3358387  1.3358387  1.61042427]


Precompute matrix elements of expansion terms in Hermite basis, i.e., $\langle\mathcal{H}_i|r_\lambda^{t_\lambda}|\mathcal{H}_j\rangle$, $\langle\mathcal{H}_i|r_\lambda^{t_\lambda}|\partial_{r_\lambda}\mathcal{H}_j\rangle$, and $\langle\partial_{r_\lambda}\mathcal{H}_i|r_\lambda^{t_\lambda}|\partial_{r_\lambda}\mathcal{H}_j\rangle$, for three internal valence coordinates $\lambda =1..3$.

In [17]:
no_bas = [24] * ncoo #Number of primitive basis functions
npoints = [80] * ncoo #Number of quadrature points

prim_me = []
prim_dme = []
prim_d2me = []

for icoo in range(ncoo):
    x, w = hermgauss(npoints[icoo]) #Generate quadrature
    w /= np.exp(-(x**2))            #Divide by weight function, because it is a part of the basis
    r = x_to_r_map(x, icoo)         #Obtain r (coordinate values) from x (quadrature values)
    dr = r - r0[icoo]               #Displacement coordinate (subtraction of equilibrium geometry, r0)
    r_pow = dr[:, None] ** powers[icoo][None, :] #Generate powers of r
    psi = hermite(x, np.arange(no_bas[icoo] + 1))#Primitive basis functions
    dpsi = hermite_deriv(x, np.arange(no_bas[icoo] + 1)) / jac_x_to_r_map(x, icoo)[:, None] #Derivative of primitive basis functions
    me = jnp.einsum("gi,gt,gj,g->tij", psi, r_pow, psi, w)     #For Potential and p_i G_ij(q_k)p_j (averaging over q_k).
    dme = jnp.einsum("gi,gt,gj,g->tij", psi, r_pow, dpsi, w)   #For p_i G_ij p_j terms, j != i
    d2me = jnp.einsum("gi,gt,gj,g->tij", dpsi, r_pow, dpsi, w) #For p_i G_ii p_i terms (j=i)
    prim_me.append(me)
    prim_dme.append(dme)
    prim_d2me.append(d2me)

Solve contracted problems with basis |i>|0>|0> with i = 0...no_bas for each mode (averaging over lowest basis function for inactive modes).

In [18]:
contr_vec = []
for icoo in range(ncoo):
    bas_ind = [
        np.arange(no_bas[icoo] + 1) if icoo_ == icoo else np.arange(1)
        for icoo_ in range(ncoo)
    ]
    bas_ind, bas_mind = next(generate_prod_ind(bas_ind))
    
    me = np.prod(
        [
            prim_me[icoo_][
                np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
            ]
            for icoo_ in range(ncoo)
        ],
        axis=0,
    )

    d2me = np.prod(
        [
            (
                prim_d2me[icoo_][
                    np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
                ]
                if icoo_ == icoo
                else prim_me[icoo_][
                    np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
                ]
            )
            for icoo_ in range(ncoo)
        ],
        axis=0,
    )

    vme = me.T @ poten_coefs
    gme = d2me.T @ Gmat_coefs[:, icoo, icoo]
    hme = 0.5 * gme + vme
    e, v = np.linalg.eigh(hme)
    print(icoo, e[0], e - e[0])
    contr_vec.append(v)

0 2361.7950574890965 [     0.           2627.8331008    5165.68583626   7629.77902022
  10056.05644398  12496.83848234  15001.60349714  17601.70795403
  20311.20249726  23134.23315601  26070.907911    29121.93829635
  32293.35520577  35602.00159154  39046.9587134   42614.21730987
  46312.26857337  50361.50583571  54604.12115941  59880.29432958
  70404.08624295  90155.41843916 123427.89263171 180304.67102473
 289652.06656253]
1 2361.7950575912123 [     0.           2627.8331008    5165.68583624   7629.77902015
  10056.05644375  12496.83848183  15001.60349622  17601.70795262
  20311.2024953   23134.23315346  26070.90790783  29121.93829252
  32293.35520127  35602.0015863   39046.9587073   42614.21730277
  46312.26856538  50361.50582748  54604.12115068  59880.29431964
  70404.08623126  90155.41842511 123427.89261331 180304.67099721
 289652.06651237]
2 2026.5007149073547 [    0.          1214.18285843  2422.52098539  3624.57606606
  4819.78075734  5195.07909558  6007.22907106  7185.25403922

Transform primitive matrix elements to the contracted basis

In [19]:
contr_me = [contr_vec[icoo].T @ prim_me[icoo] @ contr_vec[icoo] for icoo in range(ncoo)]
contr_dme = [contr_vec[icoo].T @ prim_dme[icoo] @ contr_vec[icoo] for icoo in range(ncoo)]
contr_d2me = [ contr_vec[icoo].T @ prim_d2me[icoo] @ contr_vec[icoo] for icoo in range(ncoo)]

Final solution 

In [20]:
pmax = 16
poly_coefs = np.array([2, 2, 1])

bas_ind = [np.arange(no_bas[icoo] + 1) for icoo in range(ncoo)]
bas_ind, bas_mind = next(
    generate_prod_ind(bas_ind, select=lambda ind: np.sum(ind * poly_coefs) <= pmax)
) #Polyad truncation: np.sum(ind * poly_coefs) <= pmax
print("polyad contraction number:", pmax)
print("total number of basis products:", len(bas_ind))

me = np.prod(
    [
        contr_me[icoo_][
            np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
        ]
        for icoo_ in range(ncoo)
    ],
    axis=0,
)

vme = me.T @ poten_coefs

gme = 0
fac = {}
for icoo in range(ncoo):
    for jcoo in range(ncoo):
        fac[icoo] = 1
        fac[jcoo] = -1

        if icoo != jcoo:
            dme = np.prod(
                [
                    (
                        fac[icoo_]
                        * contr_dme[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                        if icoo_ == icoo or icoo_ == jcoo
                        else contr_me[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                    )
                    for icoo_ in range(ncoo)
                ],
                axis=0,
            )
        else:
            dme = np.prod(
                [
                    (
                        contr_d2me[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                        if icoo_ == icoo
                        else contr_me[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                    )
                    for icoo_ in range(ncoo)
                ],
                axis=0,
            )
        gme += dme.T @ Gmat_coefs[:, icoo, jcoo]


hme = 0.5 * gme + vme
e, v = np.linalg.eigh(hme)
print(e[0], e - e[0])

polyad contraction number: 16
total number of basis products: 285
3300.812812486518 [    0.          1182.65648813  2354.03454468  2614.96594692
  2629.1741213   3349.08891174  3513.71735093  3779.97121436
  3790.16617169  4661.07048177  4933.62839273  4940.11098613
  5151.10750011  5152.99824515  5195.6605576   5245.23225739
  5508.6890239   5795.01248293  6075.454126    6078.39858205
  6295.12621333  6295.66241479  6387.75505656  6913.5366013
  6977.42613527  7184.75071492  7204.16245646  7204.67324144
  7427.19362789  7427.64376561  7519.57069209  7606.33228248
  7606.46030241  7710.41843418  7761.40986348  7788.6202203
  8012.61970144  8315.99329193  8319.88955389  8546.86690542
  8548.02489438  8639.94225595  8729.14528262  8729.44893933
  8860.44477557  8889.18457458  8907.68857967  9082.6095362
  9137.9931939   9411.33932732  9418.28655872  9653.64009024
  9655.25981375  9660.72786037  9747.83379827  9840.64663429
  9841.0238575  10005.95503917 10013.63755782 10013.89180121
 100